In [1]:
from WickContractions.wick.contract import *
from WickContractions.laph.diagram import *

from utilities import *

from IPython.display import display, Math

pprint = lambda o : display(Math(str(o)))

## Operators

At the moment we are hardcoding in the operators.  We will want to read in a dictionary of spin matrices, as well as isospin flavor combinations.

In [2]:
# TODO get input from operator code for Isospin/momentum combinations.

In [4]:
# load definition of the building blocks of the operators.
from udud_elementals import *


gammas = [i for i in range(2)]

gammaName = lambda i : "\\Gamma^{{ {} }}".format(i)

aOps=[]
cOps=[]
for g in gammas:
    aOps.append(Operator([baryonSink(1,['u','d','u','d'],'x_0',gammaName(g))]))
    cOps.append(Operator([baryonSource(1,['d','u','d','u'], 'x_1',gammaName(g))]))

## Wick Contractions

Now we run the wick contractions giving us essentially a list of tensor contractions to be performed.

In [5]:
diagrams=[]
for aOp in aOps:
    for cOp in cOps:
        for d in contract(aOp, cOp).diagrams:
            diagrams.append(d)
print("Need {} diagrams".format(len(diagrams)))

Need 16 diagrams


In [6]:
laphDiagrams = [LDiagram(d) for d in diagrams]
simplify_laphDiagrams(laphDiagrams)
for d in laphDiagrams:
    pprint(d)

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

...

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

In [7]:
allBaryonTensors = find_all_baryon_tensors(laphDiagrams)

In [8]:
allBaryonTensors

{'B(x_1,t_i,\\Gamma^{ 0 })': 0,
 'B^*(x_0,t_i,t_f,\\Gamma^{ 0 })': 1,
 'B(x_1,t_i,\\Gamma^{ 1 })': 2,
 'B^*(x_0,t_i,t_f,\\Gamma^{ 1 })': 3}

In [27]:
def cppArrayPrint(lst):
    return str(lst).replace('[','{').replace(']','}')


def create_diagram_gpu_file(laphDiagrams, allBaryonTensors):
    print("file header")
    for d in laphDiagrams:
        for blocks,contraction in diagram_as_graph(d,allBaryonTensors).items():
            blocks=blocks.split(',')

            block0 = int(blocks[0])
            block1 = int(blocks[1])
            btype0 = ""
            btype1 = ""
            if block0 % 2 == 0:
                btype0 = "bsinks"
                block0=block0//2
            else:
                btype0 = "bprops"
                block0=(block0-1)//2

            if block1 % 2 == 0:
                btype1 = "bsinks"
                block1 = block1//2
            else:
                btype1 = "bprops"
                block1 = (block1-1)//2

            print("diagrams.push_back(Diagram({}[{}],{}[{}],std::vector<std::vector<int>>{}));".format(btype0,block0,btype1,block1,cppArrayPrint(contraction)))

    print("fild footer")
        

create_diagram_gpu_file(laphDiagrams,allBaryonTensors)
#for d in laphDiagrams:
    #print(diagram_as_graph(d,allBaryonTensors))
    

file header
diagrams.push_back(Diagram(bprops[0],bsinks[0],std::vector<std::vector<int>>{{3, 0}, {2, 3}, {1, 2}, {0, 1}}));
diagrams.push_back(Diagram(bprops[0],bsinks[0],std::vector<std::vector<int>>{{3, 2}, {2, 3}, {1, 0}, {0, 1}}));
diagrams.push_back(Diagram(bprops[0],bsinks[0],std::vector<std::vector<int>>{{3, 0}, {2, 1}, {1, 2}, {0, 3}}));
diagrams.push_back(Diagram(bprops[0],bsinks[0],std::vector<std::vector<int>>{{3, 2}, {2, 1}, {1, 0}, {0, 3}}));
diagrams.push_back(Diagram(bprops[0],bsinks[1],std::vector<std::vector<int>>{{3, 0}, {2, 3}, {1, 2}, {0, 1}}));
diagrams.push_back(Diagram(bprops[0],bsinks[1],std::vector<std::vector<int>>{{3, 2}, {2, 3}, {1, 0}, {0, 1}}));
diagrams.push_back(Diagram(bprops[0],bsinks[1],std::vector<std::vector<int>>{{3, 0}, {2, 1}, {1, 2}, {0, 3}}));
diagrams.push_back(Diagram(bprops[0],bsinks[1],std::vector<std::vector<int>>{{3, 2}, {2, 1}, {1, 0}, {0, 3}}));
diagrams.push_back(Diagram(bsinks[0],bprops[1],std::vector<std::vector<int>>{{3, 2}, {2, 1},

In [13]:
for d in laphDiagrams:
    print(diagram_as_graph(d,allBaryonTensors))

{'1,0': [[3, 0], [2, 3], [1, 2], [0, 1]]}
{'1,0': [[3, 2], [2, 3], [1, 0], [0, 1]]}
{'1,0': [[3, 0], [2, 1], [1, 2], [0, 3]]}
{'1,0': [[3, 2], [2, 1], [1, 0], [0, 3]]}
{'1,2': [[3, 0], [2, 3], [1, 2], [0, 1]]}
{'1,2': [[3, 2], [2, 3], [1, 0], [0, 1]]}
{'1,2': [[3, 0], [2, 1], [1, 2], [0, 3]]}
{'1,2': [[3, 2], [2, 1], [1, 0], [0, 3]]}
{'0,3': [[3, 2], [2, 1], [1, 0], [0, 3]]}
{'0,3': [[3, 2], [2, 3], [1, 0], [0, 1]]}
{'0,3': [[3, 0], [2, 1], [1, 2], [0, 3]]}
{'0,3': [[3, 0], [2, 3], [1, 2], [0, 1]]}
{'3,2': [[3, 0], [2, 3], [1, 2], [0, 1]]}
{'3,2': [[3, 2], [2, 3], [1, 0], [0, 1]]}
{'3,2': [[3, 0], [2, 1], [1, 2], [0, 3]]}
{'3,2': [[3, 2], [2, 1], [1, 0], [0, 3]]}
